In [ ]:
pip install psycopg2 pandas requests

In [1]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import requests

languageCode = 'fr'
t_host = "DRAK" # either "localhost", a domain name, or an IP address.
t_port = "5435" # default postgres port
t_dbname = "postgres"
t_user = "postgres"
t_pw = "postgres"

input_data = None

with psycopg2.connect("host='{}' port={} dbname={} user={} password={}".format(t_host, t_port, t_dbname, t_user, t_pw)) as conn:
    
    sql_select_all = '''SELECT sub.id,  sub.geom_x,  sub.geom_y, sub.geom, sub.siren, sub.siret, sub.statutdiffusionetablissement, sub.adress, sub.adress_without_postal_code,
                            sub.libellevoieetablissement, sub.libellecommuneetablissement, 
                            COALESCE(sub.enseigne1etablissement, sub.enseigne2etablissement, sub.enseigne3etablissement) as enseigneetablissement,
                            COALESCE(sub.denominationusuelleetablissement, sub.enseigne1etablissement, sub.enseigne2etablissement, sub.enseigne3etablissement) as denominationusuelleetablissement 
                            FROM (SELECT id, geom, siren, siret, statutdiffusionetablissement, 
                                UPPER(CONCAT_WS(' ', NULLIF(numerovoieetablissement::text, '[ND]'), NULLIF(typevoieetablissement::text, '[ND]'), NULLIF(libellevoieetablissement::text, '[ND]'), NULLIF(codepostaletablissement::text, '[ND]'), NULLIF(libellecommuneetablissement::text, '[ND]'))) as adress,
                                UPPER(CONCAT_WS(' ', NULLIF(numerovoieetablissement::text, '[ND]'), NULLIF(typevoieetablissement::text, '[ND]'), NULLIF(libellevoieetablissement::text, '[ND]'), NULLIF(libellecommuneetablissement::text, '[ND]'))) as adress_without_postal_code,
                                NULLIF(UPPER(libellevoieetablissement), '[ND]') as libellevoieetablissement, 
                                NULLIF(UPPER(libellecommuneetablissement), '[ND]') as libellecommuneetablissement, 
                                NULLIF(denominationusuelleetablissement, '[ND]') as denominationusuelleetablissement,
                                NULLIF(enseigne1etablissement, '[ND]') as enseigne1etablissement,
                                NULLIF(enseigne2etablissement, '[ND]') as enseigne2etablissement,
                                NULLIF(enseigne3etablissement, '[ND]') as enseigne3etablissement,
                                ST_X(ST_GeomFromEWKT(geom)) as geom_x,
                                ST_Y(ST_GeomFromEWKT(geom)) as geom_y
                               FROM siren.siren_archis order by id ASC) sub;'''
    input_data = pd.read_sql_query(sql_select_all, conn)
    



    

/tmp/ipykernel_3155/2221019325.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  input_data = pd.read_sql_query(sql_select_all, conn)


In [2]:
import re
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup
import datetime

headers_websites = {"Referer":None,
                    "Sec-Ch-Ua":"'\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"'",
                    "Sec-Ch-Ua-Mobile":"?0",
                    "Sec-Ch-Ua-Platform": "Windows",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
                   }


class main_email_scrapper:
    
    def __init__(self, original_url):
        self.root_original_url = original_url
        self.original_url = original_url
        

            
        
            
        self.start_time = datetime.datetime.now()
        self.end_time =  self.start_time + datetime.timedelta(seconds=5)
        # to save scraped urls
        self.scraped = set()
        self.steps = 0
        # to save fetched emails
        self.emails = set() 
        
        self.scrap_emails_from_url(original_url)
        
        
    def finished(self):
        return list(self.emails)
          
    def scrap_emails_from_url(self, url):
        if url.find('///') < 0 and url.find('tel:') < 0:
            if self.steps < 2:
                if datetime.datetime.now() >= self.end_time and len(list(self.emails)) < 1:
                    self.end_time =  self.end_time + datetime.timedelta(seconds=5)
                    self.steps += 1
                if datetime.datetime.now() <= self.end_time:


                    if url not in self.scraped:
                        self.scraped.add(url)

                        parts = urlsplit(url)

                        base_url = "{0.scheme}://{0.netloc}".format(parts)

                        if '/' in parts.path:
                            path = url[:url.rfind('/')+1]
                        else:
                            path = url

                        print(path)

                        print("Crawling URL %s" % url)
                        
                        headers_websites["Referer"] = url
                        www_added = False
                        try_https = False
                        
                        try:
                            if url.upper().find('WWW.') < 0 and not www_added:
                                url = url.replace('//','//www.')
                                self.original_url = self.root_original_url.replace('//','//www.')
                                www_added = True
                            else:
                                self.original_url = self.root_original_url
                            if url.upper().find('HTTP//.') >= 0 and not try_https:
                                url.replace('http//','https//')
                                self.original_url = self.root_original_url.replace('http//','https//')
                                try_https = True
                            else:
                                self.original_url = self.original_url
                                    
                            r = requests.get(url, headers=headers_websites)
                            r.encoding = r.apparent_encoding
                        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, UnicodeDecodeError):
                            try:
                                if url.upper().find('WWW.') < 0 and not www_added:
                                    url = url.replace('//','//www.')
                                    self.original_url = self.root_original_url.replace('//','//www.')
                                    www_added = True
                                else:
                                    self.original_url = self.root_original_url
                                if url.upper().find('HTTP//.') >= 0 and not try_https:
                                    url.replace('http//','https//')
                                    self.original_url = self.root_original_url.replace('http//','https//')
                                    try_https = True
                                else:
                                    self.original_url = self.original_url
                                    
                                r = requests.get(url, headers=headers_websites)
                                r.encoding = r.apparent_encoding
                            except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, UnicodeDecodeError):
                                pass

                                
                            
                        try:
                            new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-zA-Z]+", r.text if r.text else '', re.I))
                            new_emails_updated = set([new_email for new_email in list(new_emails) if new_email.upper().find('.JPG') < 0 and new_email.upper().find('.JPEG') < 0 and new_email.upper().find('.PNG') < 0 and new_email.upper().find('.WEBP') < 0 and new_email.upper().find('.MP4') < 0])
                            self.emails.update(new_emails_updated)
                        except UnboundLocalError:
                            pass
                        
                        soup = None
                        current_urls = set()
                        new_emails_set = set() 
                        
                        try:
                            soup = BeautifulSoup(r.text)
                        except UnboundLocalError:
                            soup = None
                            
                        if soup != None:
                            for anchor in soup.find_all("a"):

                                if "href" in anchor.attrs:
                                    link = anchor.attrs["href"]
                                else:
                                    link = ''

                                if (link.upper().find('.JPG') < 0 and link.upper().find('.JPEG') < 0 and link.upper().find('.PNG') < 0 and link.upper().find('.WEBP') < 0 and link.upper().find('.MP4') < 0) and link.find('javascript:') < 0:

                                    if link.startswith('mailto:'):
                                        email = link.split(':')[1]
                                        self.emails.add(email)
                                    else:
                                        if link != self.original_url:
                                            if link.startswith('/') and not link.startswith('#'):
                                                url = path + link
                                            elif link.startswith(self.original_url) and link.upper().find('#') < 0:
                                                url = link

                                            current_urls.add(url)
                                  
                            if len(list(current_urls)) > 0:
                                current_urls_list = list(current_urls)
                                res = [i for i in current_urls_list if 'CONTACT' in i.upper()]

                                if len(res) > 0:
                                    for res_url in res:
                                        self.scrap_emails_from_url(res_url)
                                for cur_url in current_urls_list:
                                    self.scrap_emails_from_url(cur_url)
                                            

                      

In [ ]:
import json


headers_google_api = {"Content-type": "application/json", "X-Goog-Api-Key": "YOUR_GOOGLE_API_TOKEN", "X-Goog-FieldMask": "*"}



key = 'YOUR_GOOGLE_API_TOKEN'

for index, data in input_data.iterrows():
    input_name = None 
    mail_scrapper = None 
    all_data_results = []
    
    if data['enseigneetablissement'] is not None:
        input_name = data['enseigneetablissement']
    elif data['denominationusuelleetablissement'] is not None:
        input_name = data['denominationusuelleetablissement']
    else:
        input_name = 'architecte'
    
    input_name = input_name.replace(' ', '%2C') + '%2C' + data["adress"].replace(' ', '%2C')
        
    url = '''https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword={name}&location={coordinates}&radius={radius}&key={key}'''.format(name=input_name,
                                                                                                                                                    coordinates=str(data['geom_y']) + '%2C' + str(data['geom_x']),
                                                                                                                                                    radius=500,
                                                                                                                                                    key=key
                                                                                                                                                    )
    r = requests.get(url)
    
    
    if r.status_code == 200 and r.text != '' and r.json() and str(r.json()) != '{}':
        request_results = r.json()
        if request_results["status"] != "ZERO_RESULTS":
            



            data_adress_matching_array = data['adress_without_postal_code'].replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip().split()


            for y, place_result in enumerate(request_results["results"]):
                data_result = {"ranking": {"adress_indice":0, "same_adress":False, "name_in_base": False, "same_name":False, "result_name":""}, "place_result": place_result}




                if 'vicinity' in place_result.keys():

                    adress_place_result = place_result['vicinity'].replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip()

  
                    if all(x in adress_place_result for x in data_adress_matching_array) is True:
                        data_result["ranking"]["same_adress"] = True
                    found = 0
                    for string in data_adress_matching_array:
                        if string in adress_place_result:
                            found += 1
                    data_result["ranking"]["adress_indice"] = found


                if 'name' in place_result.keys():

                    data_result["ranking"]["result_name"] = place_result['name'].strip()


                    if data['enseigneetablissement'] is not None:           
                        name_place_compare = data['enseigneetablissement'].replace(' ',' ').replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip()
                        if name_place_compare == place_result['name'].replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip():
                           data_result["ranking"]["same_name"] = True
                    else:
                        data_result["ranking"]["name_in_base"] = False

                    if data_result["ranking"]["same_name"] is False:    
                        if data['denominationusuelleetablissement'] is not None:
                            name_place_compare = data['denominationusuelleetablissement'].replace(' ',' ').replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip()
                            if name_place_compare == place_result['name'].replace(',','').replace('é','e').replace('è','e').replace('â','a').replace('ô','o').upper().strip():
                               data_result["ranking"]["same_name"] = True
                        else:
                            data_result["ranking"]["name_in_base"] = False


            
                all_data_results.append(data_result)
                

            if len(all_data_results) > 0:
                row_data_results = sorted(all_data_results, key=lambda item: item["ranking"]["same_adress"])
                row_data_results = sorted(row_data_results, key=lambda item: item["ranking"]["adress_indice"], reverse=True)
                row_data_results = sorted(row_data_results, key=lambda item: item["ranking"]["same_name"])
            

                google_place_id = row_data_results[0]["place_result"]["place_id"]

                url_get_place_by_id = "https://places.googleapis.com/v1/places/{place_id}".format(place_id=google_place_id)

                r_details = requests.get(url_get_place_by_id, headers=headers_google_api)
                r_details_results = {}


                if r_details.status_code == 200 and r_details.text != '' and r_details.json() and str(r_details.json()) != '{}':
                    r_details_results = r_details.json()
                    phone = ''
                    websiteUri = ''
                    photos_names = set()
                    google_profile_url = ''
                    emails = []

                    if 'internationalPhoneNumber' in r_details_results.keys():
                        phone = r_details_results['internationalPhoneNumber']
                    elif 'nationalPhoneNumber' in r_details_results.keys():
                        phone = r_details_results['nationalPhoneNumber']

                    if  "websiteUri" in r_details_results.keys():
                        websiteUri = r_details_results['websiteUri']
                        email_scrapper = main_email_scrapper(websiteUri)
                        emails = email_scrapper.finished()

                    if  "displayName" in r_details_results.keys():
                        displayName = r_details_results['displayName']['text']



                    if 'photos' in r_details_results.keys() and len(r_details_results['photos']) > 0:
                        for photo in r_details_results['photos']:
                            if 'authorAttributions' in photo.keys() and len(photo['authorAttributions']) > 0:
                                photos_names.add((photo['authorAttributions'][0]['displayName'],photo['authorAttributions'][0]['uri']))    

                    google_profile_url_array = [photo[1] for photo in photos_names if photo[0].upper().strip() == displayName.upper().strip()]
                    if len(google_profile_url_array) > 1:
                        google_profile_url = google_profile_url_array[0]



                sql_upsert = '''INSERT INTO siren.google_place
                                    (siren_archis_fk, "result", complete_return, place_details, google_place_id, website, phone, emails, google_profile_url)
                                    VALUES ({siren_archis_fk}, '{result}'::json, '{complete_return}'::json, '{place_details}'::json, '{google_place_id}'::text, '{website}'::text, '{phone}'::text, '{emails}'::text[], '{google_profile_url}'::text)
                                    ON CONFLICT (siren_archis_fk)
                                    DO NOTHING;'''.format(siren_archis_fk=data['id'], 
                                                          result=json.dumps(row_data_results[0]).replace("'", "''"), 
                                                          complete_return=json.dumps(row_data_results).replace("'", "''"),
                                                          place_details=json.dumps(r_details_results).replace("'", "''"),
                                                          google_place_id=google_place_id,
                                                          website=websiteUri,
                                                          phone=phone,
                                                          emails='{' + ','.join(emails).replace("'", "''") + '}' if len(emails) > 0 else '{NULL}',
                                                          google_profile_url=google_profile_url
                                                         )    


                print(index, r.status_code, google_place_id)

                with psycopg2.connect("host='{}' port={} dbname={} user={} password={}".format(t_host, t_port, t_dbname, t_user, t_pw)) as conn:                       
                    cur = conn.cursor()
                    cur.execute(sql_upsert)
                    conn.commit()
                    cur.close()

           
        